In [1]:
import pandas as pd
import re
from io import StringIO
import numpy as np
import base64

In [2]:
df = pd.read_csv('labelled_enron.csv')
df

,column_a,message_id,date,from,to,subject,x_from,x_to,x_cc,x_bcc,...,cat_10_level_1,cat_10_level_2,cat_10_weight,cat_11_level_1,cat_11_level_2,cat_11_weight,cat_12_level_1,cat_12_level_2,cat_12_weight,labeled
0,379,<9831685.1075855725804.JavaMail.evans@thyme>,2001-03-15T14:45:00,frozenset({'phillip.allen@enron.com'}),frozenset({'todd.burke@enron.com'}),Re: Confidential Employee Information/Lenhart,Phillip K Allen,Todd Burke,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,381,<21041312.1075855725847.JavaMail.evans@thyme>,2001-03-15T14:11:00,frozenset({'phillip.allen@enron.com'}),frozenset({'kim.bolton@enron.com'}),RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Phillip K Allen,Kim Bolton,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,2139,<5907100.1075858639941.JavaMail.evans@thyme>,2001-06-20T17:04:51,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,2140,<26625142.1075858639964.JavaMail.evans@thyme>,2001-06-20T17:09:00,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'matthew.le...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Lenhart, Matthew </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,2232,<19730598.1075858642129.JavaMail.evans@thyme>,2001-08-09T12:30:58,frozenset({'k..allen@enron.com'}),"frozenset({'matt.smith@enron.com', 'm..tholt@e...",FW: Western Wholesale Activities - Gas & Power...,"Allen, Phillip K. </O=ENRON/OU=NA/CN=RECIPIENT...","Smith, Matt </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1697,499650,<18361957.1075861368310.JavaMail.evans@thyme>,2001-11-05T13:42:45,frozenset({'louise.kitchen@enron.com'}),"frozenset({'greg.whalley@enron.com', 'john.lav...",Confidential Due Diligence Info - GS,"Kitchen, Louise </O=ENRON/OU=NA/CN=RECIPIENTS/...","Whalley, Greg </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1698,501042,<29049822.1075842045435.JavaMail.evans@thyme>,2001-06-13T14:00:00,frozenset({'steven.kean@enron.com'}),"frozenset({'greg.whalley@enron.com', 'kenneth....",California's dim bulbs,Steven J Kean,"Kenneth Lay, Jeff Skilling, Andrew S Fastow, G...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1699,507376,<205897.1075861997314.JavaMail.evans@thyme>,2001-11-13T22:20:01,frozenset({'jeanie.slone@enron.com'}),frozenset({'mark.whitt@enron.com'}),RE: confidential ee info,"Slone, Jeanie </O=ENRON/OU=NA/CN=RECIPIENTS/CN...","Whitt, Mark </O=ENRON/OU=NA/CN=RECIPIENTS/CN=M...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1700,509660,<23743848.1075863311776.JavaMail.evans@thyme>,2001-07-11T15:29:22,frozenset({'legalonline-compliance@enron.com'}),"frozenset({'williams@mailman.enron.com', 'bwil...",Confidential Information and Securities Trading,Office of the Chairman - Enron Wholesale Servi...,"WILLIAMS@mailman.enron.com, WILLIAM <bwillia5@...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [3]:
df=df[~df['content'].isna()]
df=df[df['labeled']==True]
col=['message_id','subject','content','cat_1_level_2']
df=df.loc[:,col]
df.head()

,message_id,subject,content,cat_1_level_2
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,1.0
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,3.0
2,<5907100.1075858639941.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Hawkins, Bern...",4.0
3,<26625142.1075858639964.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Perrino, Dave...",4.0
4,<19730598.1075858642129.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Comnes, Alan ...",4.0


In [4]:
# Create a new 'label' column based on 'cat_1_level_2'
df['label'] = df['cat_1_level_2'].apply(lambda x: 1.0 + (x / 10.0))

# Map 'label' values to the desired labels
label_mapping = {
    1.1: 'Company Business/Strategy',
    1.2: 'Purely Personal',
    1.3: 'Personal but in professional context',
    1.4: 'Logistic Arrangements',
    1.5: 'Employment arrangements',
    1.6: 'Document editing/checking/collaboration',
    1.7: 'Empty message (due to missing attachment)',
    1.8: 'Empty message'
}

df['label'] = df['label'].map(label_mapping)
df.drop(columns=['cat_1_level_2'], inplace=True)
df

,message_id,subject,content,label
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,Company Business/Strategy
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,Personal but in professional context
2,<5907100.1075858639941.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Hawkins, Bern...",Logistic Arrangements
3,<26625142.1075858639964.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Perrino, Dave...",Logistic Arrangements
4,<19730598.1075858642129.JavaMail.evans@thyme>,FW: Western Wholesale Activities - Gas & Power...,"-----Original Message----- From: Comnes, Alan ...",Logistic Arrangements
...,...,...,...,...
1697,<18361957.1075861368310.JavaMail.evans@thyme>,Confidential Due Diligence Info - GS,There is a lot of confidential information her...,Company Business/Strategy
1698,<29049822.1075842045435.JavaMail.evans@thyme>,California's dim bulbs,You may have seen this when it first came out ...,Company Business/Strategy
1699,<205897.1075861997314.JavaMail.evans@thyme>,RE: confidential ee info,"Mark, Can you email me or fax me any written d...",Employment arrangements
1700,<23743848.1075863311776.JavaMail.evans@thyme>,Confidential Information and Securities Trading,"To:WILLIAMS, WILLIAM Email:bwillia5@enron.com ...",Company Business/Strategy


In [5]:
df['content']=df['content'].replace({'\n':' ','\t':' '},regex=True)
df['content']=df['content'].replace({',':'',},regex=True)
df['content']=df['content'].replace({';':'',},regex=True)

df['content']=df['content'].replace({'"':'',},regex=True)
df['content']=df['content'].replace({'"':''},regex=True)

df['subject']=df['subject'].replace({"'":""},regex=True)
df['subject']=df['subject'].replace({"'":""},regex=True)

df=df.loc[(~df['subject'].str.startswith('FW:',na=False))  ,:]
df.loc[(~df['subject'].str.startswith('FW:',na=False))  ,:]
df.loc[df['content'].str.contains('-----Original Message-----'),'content']=df.loc[df['content'].str.contains('-----Original Message-----'),'content'].str.extract(r'([\s\S]*?)'+r'-----Original Message-----',expand=False)
df.loc[df['content'].str.contains('----- Forwarded by'),'content']=df.loc[df['content'].str.contains('----- Forwarded by'),'content'].str.extract(r'([\s\S]*?)'+r'----- Forwarded by',expand=False)
df.loc[df['content'].str.contains('---------------------- Forwarded by'),'content']=df.loc[df['content'].str.contains('---------------------- Forwarded by'),'content'].str.extract(r'([\s\S]*?)'+r'---------------------- Forwarded by',expand=False)
df.loc[df['content'].str.contains('-------------------------- Sent from my BlackBerry'),'content']=df.loc[df['content'].str.contains('-------------------------- Sent from my BlackBerry'),'content'].str.extract(r'([\s\S]*?)'+r'-------------------------- Sent from my BlackBerry',expand=False)

df.rename(columns={'content': 'body', 'label': 'category'}, inplace=True)
df.to_csv('simplified_labeled_enron.csv', index=False)
df.head()

/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_78398/2581723703.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'content': 'body', 'label': 'category'}, inplace=True)


,message_id,subject,body,category
0,<9831685.1075855725804.JavaMail.evans@thyme>,Re: Confidential Employee Information/Lenhart,I also need to know the base salaries of Jay R...,Company Business/Strategy
1,<21041312.1075855725847.JavaMail.evans@thyme>,RE: PERSONAL AND CONFIDENTIAL COMPENSATION INF...,Thanks for the information. It would be helpfu...,Personal but in professional context
5,<21261996.1075858638025.JavaMail.evans@thyme>,Re: Western Wholesale Activities - Gas & Power...,Can you guys coordinate to make sure someone l...,Logistic Arrangements
6,<20399547.1075857614321.JavaMail.evans@thyme>,Re: confidential employee information-dutch qu...,thx Jeanie Slone 12/19/2000 04:51 PM To: John ...,Employment arrangements
7,<860767.1075849626951.JavaMail.evans@thyme>,Re: HP -- confidential internal document,This is an excellent update. Thanks for puttin...,Company Business/Strategy


In [6]:
# Split into train and test dataset
original_df = pd.read_csv('simplified_labeled_enron.csv')

# Shuffle the rows randomly
shuffled_df = original_df.sample(frac=1, random_state=42)  # Use a fixed random_state for reproducibility

# Split the DataFrame into two parts
train_df = shuffled_df.iloc[:1400]
test_df = shuffled_df.iloc[1400:]

# Save the two split DataFrames to separate CSV files
train_df.to_csv('simplified_labeled_enron_train.csv', index=False)
test_df.to_csv('simplified_labeled_enron_test.csv', index=False)

In [7]:
# function to create tuning prompts
def create_train_prompt(row):
    category = str(row['category'])  
    subject = str(row['subject'])    
    body = str(row['body'])          
    
    # categorization train prompt with placeholders for subject, body, and category
    categorization_train_prompt = (
        "<s>[INST] <<SYS>> You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n"
        "1. 'Company Business/Strategy'\n"
        "2. 'Purely Personal'\n"
        "3. 'Personal but in a professional context'\n"
        "4. 'Logistic Arrangements'\n"
        "5. 'Employment arrangements'\n"
        "6. 'Document editing/checking/collaboration'\n"
        "7. 'Empty message (due to missing attachment)'\n"
        "8. 'Empty message'\n"
        "Please provide only one category (e.g., 'Purely Personal'). <</SYS>>\n"
        "\n"
        "Subject:: {{subject}}\n"
        "Body:: {{body}}\n"
        "\n"
        "What should this email be categorized as?\n" 
        "[/INST] {{category}} <s>"
    )
    
    # replacing placeholders with actual values
    prompt = categorization_train_prompt.replace("{{subject}}", subject)
    prompt = prompt.replace("{{body}}", body)
    prompt = prompt.replace("{{category}}", category)
    
    return prompt

train_df['prompts'] = train_df.apply(create_train_prompt, axis=1)
train_df['prompts'][0]


/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_78398/2869553928.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['prompts'] = train_df.apply(create_train_prompt, axis=1)


"<s>[INST] <<SYS>> You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n1. 'Company Business/Strategy'\n2. 'Purely Personal'\n3. 'Personal but in a professional context'\n4. 'Logistic Arrangements'\n5. 'Employment arrangements'\n6. 'Document editing/checking/collaboration'\n7. 'Empty message (due to missing attachment)'\n8. 'Empty message'\nPlease provide only one category (e.g., 'Purely Personal'). <</SYS>>\n\nSubject:: Re: Confidential Employee Information/Lenhart\nBody:: I also need to know the base salaries of Jay Reitmeyer and Monique Sanchez. They are doing the same job as Matt.\n\nWhat should this email be categorized as?\n[/INST] Company Business/Strategy <s>"

In [8]:
# creating training dataset
prompts_df = pd.DataFrame(columns=['prompts'])
prompts_df['prompts'] = train_df['prompts'].astype(str)
prompts_df = prompts_df.drop(prompts_df[prompts_df['prompts'] == 'nan'].index)
prompts_df.reset_index()
prompts_df.to_csv('prompts_train.csv', index=False)
prompts_df

,prompts
1524,<s>[INST] <<SYS>> You are reviewing an email u...
1483,<s>[INST] <<SYS>> You are reviewing an email u...
354,<s>[INST] <<SYS>> You are reviewing an email u...
668,<s>[INST] <<SYS>> You are reviewing an email u...
1422,<s>[INST] <<SYS>> You are reviewing an email u...
...,...
1109,<s>[INST] <<SYS>> You are reviewing an email u...
1266,<s>[INST] <<SYS>> You are reviewing an email u...
1183,<s>[INST] <<SYS>> You are reviewing an email u...
524,<s>[INST] <<SYS>> You are reviewing an email u...


In [9]:
prompts_df['prompts'][4]

"<s>[INST] <<SYS>> You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n1. 'Company Business/Strategy'\n2. 'Purely Personal'\n3. 'Personal but in a professional context'\n4. 'Logistic Arrangements'\n5. 'Employment arrangements'\n6. 'Document editing/checking/collaboration'\n7. 'Empty message (due to missing attachment)'\n8. 'Empty message'\nPlease provide only one category (e.g., 'Purely Personal'). <</SYS>>\n\nSubject:: Re: HP -- confidential internal document\nBody:: This is an excellent update. Thanks for putting this together. Dale/Patrick - lets regroup on how we want to move this onward. Seems like SJ's suggestion of our spending more time with Bill Dwyer is a good one. Thanks Matt Sarah-Joy Hunter@ENRON 12/12/00 02:42 PM To: Matt Harris/Enron Communications@Enron Communications cc: Patrick Tucker/Enron Communications@Enron Communications Peter Goebel/NA/Enron@Enron Da

In [10]:
# function to create testing prompts
def create_test_prompt(row): 
    subject = str(row['subject'])    
    body = str(row['body'])        
    
    # categorization test prompt with placeholders for subject and body
    categorization_test_prompt = (
        "You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n"
        "1. 'Company Business/Strategy'\n"
        "2. 'Purely Personal'\n"
        "3. 'Personal but in a professional context'\n"
        "4. 'Logistic Arrangements'\n"
        "5. 'Employment arrangements'\n"
        "6. 'Document editing/checking/collaboration'\n"
        "7. 'Empty message (due to missing attachment)'\n"
        "8. 'Empty message'\n"
        "Please provide only one category (e.g., 'Purely Personal').\n"
        "\n"
        "Subject:: {{subject}}\n"
        "Body:: {{body}}\n"
        "\n"
        "What should this email be categorized as?\n"
    )
    
    # replacing placeholders with actual values
    prompt = categorization_test_prompt.replace("{{subject}}", subject)
    prompt = prompt.replace("{{body}}", body)
    
    return prompt

test_df['input_test_prompts'] = test_df.apply(create_test_prompt, axis=1)

# creating testing dataset
prompts_df_test = pd.DataFrame(columns=['input_test_prompts', 'category'])
prompts_df_test['input_test_prompts'] = test_df['input_test_prompts'].astype(str)
prompts_df_test['category'] = test_df['category'].astype(str)
prompts_df_test = prompts_df_test.drop(prompts_df_test[prompts_df_test['input_test_prompts'] == 'nan'].index)
prompts_df_test.reset_index(drop=True, inplace=True)
prompts_df_test.to_csv('prompts_test.csv', index=False)
prompts_df_test

/var/folders/wx/1_76tj0s15gc4yxmndvw4l_00000gn/T/ipykernel_78398/84615644.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['input_test_prompts'] = test_df.apply(create_test_prompt, axis=1)


,input_test_prompts,category
0,You are reviewing an email using its subject a...,Personal but in professional context
1,You are reviewing an email using its subject a...,Logistic Arrangements
2,You are reviewing an email using its subject a...,Employment arrangements
3,You are reviewing an email using its subject a...,Company Business/Strategy
4,You are reviewing an email using its subject a...,Company Business/Strategy
...,...,...
154,You are reviewing an email using its subject a...,Document editing/checking/collaboration
155,You are reviewing an email using its subject a...,Company Business/Strategy
156,You are reviewing an email using its subject a...,Company Business/Strategy
157,You are reviewing an email using its subject a...,Company Business/Strategy


Checking a few prompts...

In [11]:
prompts_df_test['input_test_prompts'][7]

"You are reviewing an email using its subject and body. Based on the text in its subject and body, please categorize this email into one of the following categories:\n1. 'Company Business/Strategy'\n2. 'Purely Personal'\n3. 'Personal but in a professional context'\n4. 'Logistic Arrangements'\n5. 'Employment arrangements'\n6. 'Document editing/checking/collaboration'\n7. 'Empty message (due to missing attachment)'\n8. 'Empty message'\nPlease provide only one category (e.g., 'Purely Personal').\n\nSubject:: Natural Gas Price Increase\nBody:: Please post and circulate to the team. \n\nWhat should this email be categorized as?\n"